pyltp 文档链接：https://pyltp.readthedocs.io/zh_CN/latest/api.html#id17

### 获取孟晚舟事件的标题

In [3]:
import numpy as np
import os

filename = np.array([])
for root, dirs, files in os.walk('WeChatArticles/'):
    filename = np.concatenate((filename,np.array([name.split(".")[0] for name in files])))

In [20]:
filename[0]

'315曝光名单来了，电子烟危害不比香烟小，孟晚舟在华为不开心'

### split_word

In [50]:
# -*- coding: utf-8 -*-
from pyltp import SentenceSplitter
from pyltp import Segmentor

import os

# sentence = '元芳你怎么看？我就趴窗口上看呗！'
sentence = filename[0]

sents = SentenceSplitter.split(sentence)  # 分句
print('\t'.join(sents))


LTP_DATA_DIR = 'pyltp_model/ltp_data_v3.4.0/'  # ltp模型目录的路径
cws_model_path = os.path.join(LTP_DATA_DIR, 'cws.model')  # 分词模型路径，模型名称为`cws.model`


segmentor = Segmentor()  # 初始化实例

# SentenceSplitter分句子，把段落拆分成句子；segmentor.segment分词
# 如果要使用外部词典，指定新的字典集：
segmentor.load_with_lexicon(cws_model_path, 'userdict.txt') # 加载模型，第二个参数是您的外部词典文件路径

segmentor.load(cws_model_path)  # 加载模型
words = segmentor.segment(sentence)  # 分词
print('\t'.join(words))
segmentor.release()  # 释放模型

315曝光名单来了，电子烟危害不比香烟小，孟晚舟在华为不开心
315	曝光名单	来	了	，	电子烟	危害	不	比	香烟	小	，	孟晚舟	在	华为	不	开心


pyltp 词性标注同样支持用户的外部词典。词性标注外部词典同样为一个文本文件，每行指定一个词，第一列指定单词，第二列之后指定该词的候选词性（可以有多项，每一项占一列），列与列之间用空格区分。

### 词性标注

In [51]:
import os
pos_model_path = os.path.join(LTP_DATA_DIR, 'pos.model')  # 词性标注模型路径，模型名称为`pos.model`

from pyltp import Postagger
postagger = Postagger() # 初始化实例
postagger.load(pos_model_path)  # 加载模型

postags = postagger.postag(words)  # 词性标注

print('\t'.join(postags))
postagger.release()  # 释放模型

m	n	v	u	wp	n	v	d	p	n	a	wp	nh	p	nz	d	a


### 命名实体识别

In [52]:
import os
from pyltp import NamedEntityRecognizer

LTP_DATA_DIR = 'pyltp_model/ltp_data_v3.4.0/'  # ltp模型目录的路径
ner_model_path = os.path.join(LTP_DATA_DIR, 'ner.model')  # 命名实体识别模型路径，模型名称为`pos.model`

recognizer = NamedEntityRecognizer() # 初始化实例
recognizer.load(ner_model_path)  # 加载模型

netags = recognizer.recognize(words, postags)  # 命名实体识别

print('\t'.join(netags))
recognizer.release()  # 释放模型

O	O	O	O	O	O	O	O	O	O	O	O	S-Nh	O	O	O	O


### 依存句法分析

In [53]:
par_model_path = os.path.join(LTP_DATA_DIR, 'parser.model')  # 依存句法分析模型路径，模型名称为`parser.model`

from pyltp import Parser
parser = Parser() # 初始化实例
parser.load(par_model_path)  # 加载模型

arcs = parser.parse(words, postags)  # 句法分析

print("\t".join("%d:%s" % (arc.head, arc.relation) for arc in arcs))
parser.release()  # 释放模型

2:ATT	3:SBV	0:HED	3:RAD	3:WP	7:SBV	3:COO	11:ADV	11:ADV	9:POB	7:VOB	7:WP	17:SBV	17:ADV	14:POB	17:ADV	7:COO


In [18]:
netags

### 语义角色分析

核心的语义角色为 A0-5 六种，A0 通常表示动作的施事，A1通常表示动作的影响等，A2-5 根据谓语动词不同会有不同的语义含义。其余的15个语义角色为附加语义角色，如LOC 表示地点，TMP 表示时间等。

In [54]:
srl_model_path = os.path.join(LTP_DATA_DIR, 'pisrl_win.model')  # 语义角色标注模型目录路径，模型目录为`srl`。注意该模型路径是一个目录，而不是一个文件。

from pyltp import SementicRoleLabeller
labeller = SementicRoleLabeller() # 初始化实例
labeller.load(srl_model_path)  # 加载模型


# arcs 使用依存句法分析的结果
roles = labeller.label(words, postags,arcs)  # 语义角色标注

# 打印结果
for role in roles:
    print(role.index, "".join(["%s:(%d,%d)" % (arg.name, arg.range.start, arg.range.end) for arg in role.arguments]))
labeller.release()  # 释放模型

2 A0:(0,1)
10 ADV:(7,7)ADV:(8,9)
16 A0:(12,12)LOC:(13,14)ADV:(15,15)


In [55]:
for item,i in zip(words,range(len(words))):
    print("{item},{num}".format(item=item,num=i),end="\t")

315,0	曝光名单,1	来,2	了,3	，,4	电子烟,5	危害,6	不,7	比,8	香烟,9	小,10	，,11	孟晚舟,12	在,13	华为,14	不,15	开心,16	